In [16]:
import gymnasium as gym
import tianshou as ts

env = gym.make('CartPole-v0')

C:\Users\20230921a\.conda\envs\tianshou\Lib\site-packages\gymnasium\envs\registration.py:513: DeprecationWarning: WARN: The environment CartPole-v0 is out of date. You should consider upgrading to version `v1`.
  logger.deprecation(


In [17]:
train_envs = gym.make('CartPole-v0')
test_envs = gym.make('CartPole-v0')

In [18]:
train_envs = ts.env.DummyVectorEnv([lambda: gym.make('CartPole-v0') for _ in range(10)])
test_envs = ts.env.DummyVectorEnv([lambda: gym.make('CartPole-v0') for _ in range(100)])

In [19]:
import torch, numpy as np
from torch import nn

class Net(nn.Module):
    def __init__(self, state_shape, action_shape):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(np.prod(state_shape), 128), nn.ReLU(inplace=True),
            nn.Linear(128, 128), nn.ReLU(inplace=True),
            nn.Linear(128, 128), nn.ReLU(inplace=True),
            nn.Linear(128, np.prod(action_shape)),
        )

    def forward(self, obs, state=None, info={}):
        if not isinstance(obs, torch.Tensor):
            obs = torch.tensor(obs, dtype=torch.float)
        batch = obs.shape[0]
        logits = self.model(obs.view(batch, -1))
        return logits, state

state_shape = env.observation_space.shape or env.observation_space.n
action_shape = env.action_space.shape or env.action_space.n
net = Net(state_shape, action_shape)
optim = torch.optim.Adam(net.parameters(), lr=1e-3)

In [20]:
policy = ts.policy.DQNPolicy(model=net,
                             optim=optim, 
                             action_space=env.action_space,
                             discount_factor=0.9, estimation_step=3, target_update_freq=320)

In [21]:
train_collector = ts.data.Collector(policy, train_envs, ts.data.VectorReplayBuffer(20000, 10), exploration_noise=True)
test_collector = ts.data.Collector(policy, test_envs, exploration_noise=True)

In [27]:
result = ts.trainer.OffpolicyTrainer(
    policy=policy,
    train_collector=train_collector,
    test_collector=test_collector,
    max_epoch=10, step_per_epoch=10000, step_per_collect=10,
    update_per_step=0.1, episode_per_test=100, batch_size=64,
    train_fn=lambda epoch, env_step: policy.set_eps(0.1),
    test_fn=lambda epoch, env_step: policy.set_eps(0.05),
    stop_fn=lambda mean_rewards: mean_rewards >= env.spec.reward_threshold).run()


Epoch #1:  24%|##4       | 2430/10000 [00:08<00:27, 280.36it/s, env_step=2430, len=200, n/ep=1, n/st=10, rew=200.00]                   


In [26]:
from torch.utils.tensorboard import SummaryWriter
from tianshou.utils import TensorboardLogger
writer = SummaryWriter('log/dqn')
logger = TensorboardLogger(writer)

In [31]:
result.timing

TimingStats(total_time=9.797119855880737, train_time=1.3845407962799072, train_time_collect=0.3725924491882324, train_time_update=0.7919645309448242, test_time=8.41257905960083, update_speed=1755.0945458083393)

In [32]:
obs, done = env.reset(), False
obs 

(array([-0.00965831,  0.02123608, -0.03720427, -0.02112211], dtype=float32),
 {})

In [33]:
from tianshou.data import Batch
data = Batch({'a': 4, 'b': [5, 5], 'c': '2312312'})
# the list will automatically be converted to numpy array
data.b

data.b = np.array([3, 4, 5])
print(data)





# a list of dict objects (possibly nested) will be automatically stacked
data = Batch([{'a': 0.0, 'b': "hello"}, {'a': 1.0, 'b': "world"}])
print(data)

Batch(
    a: array(4),
    b: array([3, 4, 5]),
    c: '2312312',
)
Batch(
    b: array(['hello', 'world'], dtype=object),
    a: array([0., 1.]),
)


In [34]:
data = Batch(a=np.array([[0.0, 2.0], [1.0, 3.0]]), b=[[5, -5], [1, -2]])
data[:, 1] += 1
data

Batch(
    a: array([[0., 3.],
              [1., 4.]]),
    b: array([[ 5, -4],
              [ 1, -1]]),
)

In [36]:
data_1 = Batch(a=np.array([0.0, 2.0]), b=5)
data_2 = Batch(a=np.array([1.0, 3.0]), b=-5)
data = Batch.stack((data_1, data_2))
data

Batch(
    b: array([ 5, -5]),
    a: array([[0., 2.],
              [1., 3.]]),
)

In [37]:
from tianshou.data import Collector
from tianshou.env import DummyVectorEnv, PettingZooEnv
from tianshou.policy import MultiAgentPolicyManager, RandomPolicy

from pettingzoo.classic import rps_v2

if __name__ == "__main__":
    # Step 1: Load the PettingZoo environment
    env = rps_v2.env(render_mode="human")

    # Step 2: Wrap the environment for Tianshou interfacing
    env = PettingZooEnv(env)

    # Step 3: Define policies for each agent
    policies = MultiAgentPolicyManager([RandomPolicy(), RandomPolicy()], env)

    # Step 4: Convert the env to vector format
    env = DummyVectorEnv([lambda: env])

    # Step 5: Construct the Collector, which interfaces the policies with the vectorised environment
    collector = Collector(policies, env)

    # Step 6: Execute the environment with the agents playing for 1 episode, and render a frame every 0.1 seconds
    result = collector.collect(n_episode=1, render=0.1)

TypeError: BasePolicy.__init__() missing 1 required keyword-only argument: 'action_space'